## Energy Transition Case Study

This notebook presents the base case optimization of the German electricity market as presented in the manuscript presenting the PULPO framework.

Updated by Fabian Lechtenberg, 15.04.2024.

See [this notebook](./electricity_showcase.ipynb) for additional results and functionalities of this case study, and [this notebook](./plastic_showcase.ipynb) for a more thorough demonstration of the functionalities on a plastics recycling case study.

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Demonstration 🚀</h2>
</div>

### Base Problem

In [ ]:
# Import section
import os
import sys
sys.path.append('../')

import pandas as pd
from pulpo import pulpo

In [ ]:
# Your brightway2 project, database and targeted impact category
project = "pulpo"
database = "cutoff38"
methods = {"('IPCC 2013', 'climate change', 'GWP 100a')": 1,
           "('ReCiPe Endpoint (E,A)', 'resources', 'total')": 0,
           "('ReCiPe Endpoint (E,A)', 'human health', 'total')": 0,
           "('ReCiPe Endpoint (E,A)', 'ecosystem quality', 'total')": 0,
           "('ReCiPe Midpoint (E) V1.13', 'ionising radiation', 'IRP_HE')": 0}

# Substitute with your working directory of choice
notebook_dir = os.path.dirname(os.getcwd())
directory = os.path.join(notebook_dir, 'data')

In [ ]:
# Create a PULPO object
pulpo_worker = pulpo.PulpoOptimizer(project, database, methods, directory)

In [ ]:
# Retrieve LCI data
pulpo_worker.get_lci_data()

In [ ]:
# Retrieve the electricity market
activities = ["market for electricity, high voltage"]
reference_products = ["electricity, high voltage"]
locations = ["DE"]
electricity_market = pulpo_worker.retrieve_activities(activities=activities,
                                                      reference_products=reference_products,
                                                      locations=locations)

In [ ]:
# Specify the functional unit as demand dictionary
demand = {electricity_market[0]: 1.28819e+11}

In [ ]:
# Retrieve the choices
activities = ["electricity production, lignite", 
             "electricity production, hard coal",
             "electricity production, nuclear, pressure water reactor",
             "electricity production, wind, 1-3MW turbine, onshore"]
reference_products = ["electricity, high voltage"]
locations = ["DE"]

electricity_activities = pulpo_worker.retrieve_activities(activities=activities,
                                                          reference_products=reference_products,
                                                          locations=locations)

In [ ]:
# Specify the choices dictionary
choices  = {'electricity': {electricity_activities[0]: 1e16,
                            electricity_activities[1]: 1e16,
                            electricity_activities[2]: 1e16,
                            electricity_activities[3]: 1e16}}

In [ ]:
# Instantiate and solve the problem (here with HiGHS)
instance = pulpo_worker.instantiate(choices=choices, demand=demand)
results = pulpo_worker.solve()

In [ ]:
# Summarize the results
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

In [ ]:
pulpo_worker.save_results(choices=choices, demand=demand, name='energy_transition_showcase.xlsx')

### No optimization

In [ ]:
# Instantiate and solve the problem (here with HiGHS)
instance = pulpo_worker.instantiate(choices={}, demand=demand)
results = pulpo_worker.solve()

In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

In [ ]:
pulpo_worker.save_results(choices=choices, demand=demand, name='energy_transition_showcase_noopt.xlsx')